In [30]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

In [31]:
#========
# Setup
#========
np.random.seed(928)

dates = pd.date_range("2025-12-01" , periods = 14 , freq = "D")

df = pd.DataFrame({
    "date": np.tile(dates , 2) ,
    "region": ["East"] * 14 + ["West"] * 14,
})
base = np.linspace(100 , 130 , 14)
df["sales"] = np.r_[base + np.random.normal(0 , 6 , 14) ,
                   base + 8 + np.random.normal(0 , 7 , 14)].round(0).astype(int)

df["returns"] = np.r_[np.random.poisson(4 , 14) , np.random.poisson(5 , 14)]
df.head()

ts = df.sort_values(["region" , "date"]).copy()
ts = ts.set_index("date")
ts.head()

,date,region,sales,returns
0,2025-12-01,East,110,4
1,2025-12-02,East,107,5
2,2025-12-03,East,104,7
3,2025-12-04,East,105,3
4,2025-12-05,East,106,3


,region,sales,returns
date,,,
2025-12-01,East,110,4
2025-12-02,East,107,5
2025-12-03,East,104,7
2025-12-04,East,105,3
2025-12-05,East,106,3


In [32]:
#================================
# Case 1) Simple rolling window
#================================
ts["sales_roll3_mean"] = (
    ts.groupby("region")["sales"].rolling(window = 3 , min_periods = 1)
        .mean().reset_index(level = 0 , drop = True)
)
ts.loc[: , ["region" , "sales" , "sales_roll3_mean"]].head(8).round(2)

,region,sales,sales_roll3_mean
date,,,
2025-12-01,East,110,110.00
2025-12-02,East,107,108.50
2025-12-03,East,104,107.00
2025-12-04,East,105,105.33
2025-12-05,East,106,105.00
2025-12-06,East,116,109.00
2025-12-07,East,116,112.67
2025-12-08,East,112,114.67


In [33]:
#====================================
# Case 2) Time-based rolling window
#====================================
ts["sales_roll7D_mean"] = (
    ts.groupby("region")["sales"].rolling(window = "7D" , min_periods = 1)
        .mean().reset_index(level = 0 , drop = True)
)
ts.loc[: , ["region" , "sales" , "sales_roll7D_mean"]].head(10).round(2)

,region,sales,sales_roll7D_mean
date,,,
2025-12-01,East,110,110.00
2025-12-02,East,107,108.50
2025-12-03,East,104,107.00
2025-12-04,East,105,106.50
2025-12-05,East,106,106.40
2025-12-06,East,116,108.00
2025-12-07,East,116,109.14
2025-12-08,East,112,109.43
2025-12-09,East,122,111.57


In [34]:
#==============================================
# Case 3) Edge controls: min_periods + center
#==============================================
ts["sales_roll5_center"] = (
    ts.groupby("region")["sales"].rolling(window = 5 , min_periods = 2 , center = True)
        .mean().reset_index(level = 0 , drop = True)
)
ts.loc[: , ["region" , "sales" , "sales_roll5_center"]].head(10).round(2)

,region,sales,sales_roll5_center
date,,,
2025-12-01,East,110,107.0
2025-12-02,East,107,106.5
2025-12-03,East,104,106.4
2025-12-04,East,105,107.6
2025-12-05,East,106,109.4
2025-12-06,East,116,111.0
2025-12-07,East,116,114.4
2025-12-08,East,112,117.4
2025-12-09,East,122,118.2


In [35]:
#==============================================
# Case 4) Multiple rolling stats at once (agg)
#==============================================
roll_stats = (
    ts.groupby("region")["sales"].rolling(window = 5 , min_periods = 3)
        .agg(["mean" , "std" , "min" , "max"]).reset_index(level = 0 , drop = True)
)
ts["sales_roll5_mean"] = roll_stats["mean"]
ts["sales_roll5_std"] = roll_stats["std"]
ts.loc[: , ["region" , "sales" , "sales_roll5_mean" , "sales_roll5_std"]].head(10).round(2)

,region,sales,sales_roll5_mean,sales_roll5_std
date,,,,
2025-12-01,East,110,NaN,NaN
2025-12-02,East,107,NaN,NaN
2025-12-03,East,104,107.0,3.00
2025-12-04,East,105,106.5,2.65
2025-12-05,East,106,106.4,2.30
2025-12-06,East,116,107.6,4.83
2025-12-07,East,116,109.4,6.07
2025-12-08,East,112,111.0,5.29
2025-12-09,East,122,114.4,5.90


In [36]:
#====================================
# Case 5) Rolling with custom logic
#====================================
ts["sales_roll5_p90"] = (
    ts.groupby("region")["sales"].rolling(window = 5 , min_periods = 3)
        .apply(lambda x: np.percentile(x , 90) , raw = True)
        .reset_index(level = 0 , drop = True)
)
ts.loc[: , ["region" , "sales" , "sales_roll5_p90"]].head(10).round(2)

,region,sales,sales_roll5_p90
date,,,
2025-12-01,East,110,NaN
2025-12-02,East,107,NaN
2025-12-03,East,104,109.4
2025-12-04,East,105,109.1
2025-12-05,East,106,108.8
2025-12-06,East,116,112.4
2025-12-07,East,116,116.0
2025-12-08,East,112,116.0
2025-12-09,East,122,119.6


In [37]:
#==============================================
# Case 6) Groupby rolling (per-segment trend)
#==============================================
tmp = ts.reset_index().sort_values(["region" , "date"]).reset_index(drop = True)

tmp["roll5_corr_sales_returns"] = (
    tmp.groupby("region" , group_keys = False)[["sales" , "returns"]]
        .apply(lambda g: g["sales"].rolling(window = 5 , min_periods = 4).corr(g["returns"]))
)

ts = tmp.set_index("date")

ts.loc[: , ["region" , "sales" , "returns" , "roll5_corr_sales_returns"]].head(12).round(2)

,region,sales,returns,roll5_corr_sales_returns
date,,,,
2025-12-01,East,110,4,NaN
2025-12-02,East,107,5,NaN
2025-12-03,East,104,7,NaN
2025-12-04,East,105,3,-0.41
2025-12-05,East,106,3,-0.31
2025-12-06,East,116,4,-0.22
2025-12-07,East,116,9,0.38
2025-12-08,East,112,2,0.58
2025-12-09,East,122,3,0.21
